# Imports

In [2]:
# misc
import os, time, sys
import math, random
from timeit import default_timer as timer

# load/save files
import requests
import zipfile
import csv, json, sqlite3
import joblib
import h5py

# plot
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from PIL import Image

# datascience libs
import numpy as np
import pandas as pd

try: # python
    path_ = os.path.join(os.path.dirname(os.path.abspath(__file__)), "..")
except NameError: # jupyter notebook
    path_ = os.path.dirname(os.getcwd())

# Load CSV into DataFrame

In [61]:
#df = pd.DataFrame(bbox, columns=['x1','y1','x2','y2'])
#df.to_csv(file_path, index=False)
path_datasets = os.path.join(path_, "datasets")
df_magasins = pd.read_csv(os.path.join(path_datasets, "magasins.csv"))
df_produits = pd.read_csv(os.path.join(path_datasets, "produits.csv"))
df_ventes = pd.read_csv(os.path.join(path_datasets, "ventes.csv"))


In [62]:
display(df_magasins.head(3))
display(df_produits.head(3))
display(df_ventes.head(3))

,ID Magasin,Ville,Nombre de salariés
0,1,Paris,10
1,2,Marseille,5
2,3,Lyon,8


,Nom,ID Référence produit,Prix,Stock
0,Produit A,REF001,49.99,100
1,Produit B,REF002,19.99,50
2,Produit C,REF003,29.99,75


,Date,ID Référence produit,Quantité,ID Magasin
0,2023-05-27,REF001,5,1
1,2023-05-28,REF002,3,2
2,2023-05-29,REF003,2,1


In [84]:
df_produits['Nom'].values.tolist()

['Produit A', 'Produit B', 'Produit C', 'Produit D', 'Produit E']

# Create DB

In [111]:

def create_table(name_table, path="", remove_if_exist=False):
    path_table = os.path.join(path, name_table)
    if os.path.exists(path_table):
        if remove_if_exist:
            os.remove(path_table)
        else:
            return
            
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()
    
    # magasins
    cursor.execute(
        "CREATE TABLE magasins (id INTEGER PRIMARY KEY AUTOINCREMENT, id_magasin INT, nom_ville TEXT, nombre_salaries INT)")
    # produits
    cursor.execute(
        "CREATE TABLE produits (id INTEGER PRIMARY KEY AUTOINCREMENT, nom TEXT, id_ref_produit TEXT, prix FLOAT, stock INT)")
    # ventes
    cursor.execute(
        "CREATE TABLE ventes (id INTEGER PRIMARY KEY AUTOINCREMENT, date DATE, id_ref_produit TEXT, quantite INT, id_magasin INT)")

    conn.commit()
    conn.close()


create_table("table.sqlite", path_datasets, True)

# Add new items (magasin)

In [121]:
def insert_items_to_magasin(name_table, path="", list_id_magasin=[], list_nom_villes=[], list_salaries=[]):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    if len(list_id_magasin) == 0:
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()

    for i in range(len(list_id_magasin)):
        query = """
            INSERT INTO magasins (id_magasin, nom_ville, nombre_salaries)
            SELECT
                T.*
            FROM
                (SELECT {} id_magasin, '{}' nom_ville, {} nombre_salaries) T
                    LEFT JOIN
                magasins ON magasins.id_magasin = T.id_magasin AND magasins.nom_ville = T.nom_ville
            WHERE
                magasins.id_magasin is null
            """.format(list_id_magasin[i], list_nom_villes[i], list_salaries[i])

        cursor.execute(query)
        conn.commit()

    conn.close()
    
    
insert_items_to_magasin("table.sqlite", path_datasets,
                        df_magasins['ID Magasin'].values.tolist(),
                        df_magasins['Ville'].values.tolist(),
                        df_magasins['Nombre de salariés'].values.tolist())


# Add new items (produits)

In [120]:
def insert_items_to_produits(name_table, path="", list_nom=[], list_id_ref_produit=[], list_prix=[], list_stock=[]):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    if len(list_nom) == 0:
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()

    for i in range(len(list_nom)):
        query = """
            INSERT INTO produits (nom, id_ref_produit, prix, stock)
            SELECT
                T.*
            FROM
                (SELECT '{}' nom, '{}' id_ref_produit, {} prix, {} stock) T
                    LEFT JOIN
                produits ON produits.nom = T.nom AND produits.id_ref_produit = T.id_ref_produit
            WHERE
                produits.nom IS null
            """.format(list_nom[i], list_id_ref_produit[i], list_prix[i], list_stock[i])

        cursor.execute(query)
        conn.commit()

    conn.close()
    

insert_items_to_produits("table.sqlite", path_datasets,
                        df_produits['Nom'].values.tolist(),
                        df_produits['ID Référence produit'].values.tolist(),
                        df_produits['Prix'].values.tolist(),
                        df_produits['Stock'].values.tolist())

# Add new items (ventes)

In [115]:
def insert_items_to_ventes(name_table, path="", list_date=[], list_id_ref_produit=[], list_quantite=[], list_id_magasin=[]):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    if len(list_date) == 0:
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()

    for i in range(len(list_date)):
        query = """
            INSERT INTO ventes (date, id_ref_produit, quantite, id_magasin)
            SELECT
                T.*
            FROM
                (SELECT '{}' date, '{}' id_ref_produit, {} quantite, {} id_magasin) T
                    LEFT JOIN
                ventes ON ventes.date = T.date AND ventes.id_ref_produit = T.id_ref_produit AND ventes.id_magasin = T.id_magasin
            WHERE
                ventes.date IS null
            """.format(list_date[i], list_id_ref_produit[i], list_quantite[i], list_id_magasin[i])

        cursor.execute(query)
        conn.commit()

    conn.close()

    
insert_items_to_ventes("table.sqlite", path_datasets,
                        df_ventes['Date'].values.tolist(),
                        df_ventes['ID Référence produit'].values.tolist(),
                        df_ventes['Quantité'].values.tolist(),
                        df_ventes['ID Magasin'].values.tolist())

# Show DB

In [122]:
def show_table(name_table, path=""):
    path_table = os.path.join(path, name_table)
    if not os.path.exists(path_table):
        return
    
    with sqlite3.connect(path_table) as conn:
        cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM magasins")
    rows = cursor.fetchall()
    print("\nmagasins:")
    print(rows)
    
    cursor.execute("SELECT * FROM produits")
    rows = cursor.fetchall()
    print("\nproduits:")
    print(rows)
    
    cursor.execute("SELECT * FROM ventes")
    rows = cursor.fetchall()
    print("\nventes:")
    print(rows)
    
    conn.close()
    
show_table("table.sqlite", path_datasets)


magasins:
[(1, 1, 'Paris', 10), (2, 2, 'Marseille', 5), (3, 3, 'Lyon', 8), (4, 4, 'Bordeaux', 12), (5, 5, 'Lille', 6), (6, 6, 'Nantes', 7), (7, 7, 'Strasbourg', 9)]

produits:
[(1, 'Produit A', 'REF001', 49.99, 100), (2, 'Produit B', 'REF002', 19.99, 50), (3, 'Produit C', 'REF003', 29.99, 75), (4, 'Produit D', 'REF004', 79.99, 120), (5, 'Produit E', 'REF005', 39.99, 80)]

ventes:
[(1, '2023-05-27', 'REF001', 5, 1), (2, '2023-05-28', 'REF002', 3, 2), (3, '2023-05-29', 'REF003', 2, 1), (4, '2023-05-30', 'REF004', 4, 3), (5, '2023-05-31', 'REF005', 7, 2), (6, '2023-06-01', 'REF001', 3, 4), (7, '2023-06-02', 'REF002', 6, 1), (8, '2023-06-03', 'REF003', 1, 5), (9, '2023-06-04', 'REF004', 2, 3), (10, '2023-06-05', 'REF005', 5, 6), (11, '2023-06-06', 'REF001', 4, 7), (12, '2023-06-07', 'REF002', 3, 2), (13, '2023-06-08', 'REF003', 6, 4), (14, '2023-06-09', 'REF004', 2, 1), (15, '2023-06-10', 'REF005', 8, 3), (16, '2023-06-11', 'REF001', 3, 2), (17, '2023-06-12', 'REF002', 5, 4), (18, '2023-0

# Analyse Exploratoire

In [20]:
print("\nmagasins:")
display(df_magasins)
print("\nproduits:")
display(df_produits)
print("\nventes:")
display(df_ventes)


magasins:


,ID Magasin,Ville,Nombre de salariés
0,1,Paris,10
1,2,Marseille,5
2,3,Lyon,8
3,4,Bordeaux,12
4,5,Lille,6
5,6,Nantes,7
6,7,Strasbourg,9



produits:


,Nom,ID Référence produit,Prix,Stock
0,Produit A,REF001,49.99,100
1,Produit B,REF002,19.99,50
2,Produit C,REF003,29.99,75
3,Produit D,REF004,79.99,120
4,Produit E,REF005,39.99,80



ventes:


,Date,ID Référence produit,Quantité,ID Magasin
0,2023-05-27,REF001,5,1
1,2023-05-28,REF002,3,2
2,2023-05-29,REF003,2,1
3,2023-05-30,REF004,4,3
4,2023-05-31,REF005,7,2
5,2023-06-01,REF001,3,4
6,2023-06-02,REF002,6,1
7,2023-06-03,REF003,1,5
8,2023-06-04,REF004,2,3
9,2023-06-05,REF005,5,6
